# Update Dataframe

This script has the objective to add submission and comment to the reddit dataframe

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import praw

In [2]:
reddit = praw.Reddit(client_id='2YvrfPL-6r2E7g',
                     client_secret='ClHTUejD1prIgWThqC5-IwswkM4',
                     user_agent='alexparentt')

# Import CSV's into pandas dataframes

In [5]:
df_submissions = pd.read_csv("reddit_submissions_wUTC_2018-05-03_1817.csv",
skipinitialspace=True,
header = 0,
sep = ',',
encoding='latin1')
print(df_submissions.shape)
df_submissions.reset_index() 

df_comments = pd.read_csv("reddit_comments_wUTC_2018-05-03_1817.csv",
skipinitialspace=True,
header = 0,
sep = ',',
encoding='latin1')
print(df_comments.shape)
df_comments.reset_index()

df_submissions = df_submissions.rename(columns={'id':'id'})
df_comments = df_comments.rename(columns={'id':'comment_id'})

(999, 25)
(4460, 19)


In [7]:
#df_submissions = df_submissions.drop(['Unnamed: 0'], axis=1)
#df_comments = df_comments.drop(['Unnamed: 0'], axis=1)
print(df_comments.shape)
print(df_comments.columns)

print(df_submissions.shape)
print(df_submissions.columns)

(4460, 19)
Index(['comment_id', '_submission', 'subreddit_id', 'score', 'author',
       'author_flair_text', 'archived', 'name', 'body', 'send_replies',
       'distinguished', 'created', 'created_utc', 'approved_by', 'link_id',
       'parent_id', 'ups', 'downs', 'gilded'],
      dtype='object')
(999, 25)
Index(['id', 'subreddit_id', 'title', 'score', 'author', 'author_flair_text',
       'likes', 'archived', 'is_video', 'name', 'over_18', 'pinned',
       'quarantine', 'selftext', 'send_replies', 'num_comments',
       'num_crossposts', 'distinguished', 'created', 'created_utc',
       'comment_limit', 'approved_by', 'ups', 'downs', 'gilded'],
      dtype='object')


# Get new data points

In [8]:
attributes_subm = ['id', 
                   'subreddit_id',
              'title', 
              'score', 
              'author',
             'author_flair_text',
              'likes', 
              'archived',
             'is_video',
             'name',
             'over_18',
             'pinned',    
             'quarantine',
             'selftext',
             'send_replies',
             'num_comments',
             'num_crossposts',
             'distinguished',
             'created',
                   'created_utc',
             'comment_limit',
             'approved_by',
            'ups',
            'downs',
            'gilded']

attributes_comments = ['id',
                        '_submission',
                       'subreddit_id',
                      'score',
                      'author',
                       'author_flair_text',
                       'archived',
                       'name',
                        'body',
                      'send_replies',
                    'distinguished',
                      'created',
                       'created_utc',
                    'approved_by',
                      'link_id',
                      'parent_id',
                      'ups',
                      'downs',
                      'gilded']

In [9]:
def attribute_to_df(df, obj, attribute, list_attributes):
    # Add attribute to the list
    att_value = getattr(obj, attribute)
    temp_list.append(att_value)
    
    # If all the attributes have been added to the list,
    # append the list to the dataframe
    if attribute == list_attributes[-1]:
        if len(df) > 0:
            df_temp = pd.DataFrame([temp_list])
            df = df.append(df_temp)
        if len(df) == 0:
            df = pd.DataFrame([temp_list])
    
    return df

In [10]:
num_subm = 1000
df = pd.DataFrame()
df2 = pd.DataFrame()
for idx, submission in enumerate(reddit.subreddit('CanadaPublicServants').hot(limit=num_subm)):
    for attribute in attributes_subm:
        # Initialize a list for each new object
        if attribute == attributes_subm[0]:
            temp_list = []
        df = attribute_to_df(df,submission,attribute,attributes_subm)
    
    for idx2, comment in enumerate(submission.comments):
        for attribute in attributes_comments:
            # Initialize a list for each new object
            if attribute == attributes_comments[0]:
                temp_list = []
            df2 = attribute_to_df(df2,comment,attribute,attributes_comments)

### Define column's name + column's index

In [12]:
df.columns = [attributes_subm]

df2.columns = [attributes_comments]
df2 = df2.rename(columns={'id':'comment_id'})

print(df.shape)
print(df2.shape)

(997, 25)
(4451, 19)


# Append new data points to current dataframe

In [13]:
new_submissions = pd.concat([df_submissions,df])
print(new_submissions.shape)
new_submissions = new_submissions.drop_duplicates(subset=['selftext'], keep='last')

new_comments = pd.concat([df_comments,df2])
print(new_comments.shape)
new_comments = new_comments.drop_duplicates(subset=['body'], keep='last')

print('Lenght of the old {}'.format(len(df_submissions))) 
print('Lenght of the new {}'.format(len(new_submissions))) 
print('Lenght of the old {}'.format(len(df_comments))) 
print('Lenght of the new {}'.format(len(new_comments))) 

print(df_submissions.shape), print(new_submissions.shape)
print(df_comments.shape), print(new_comments.shape)

print('{} new submissions added'.format(len(new_submissions)-len(df_submissions)))
print('{} new comments added'.format(len(new_comments)-len(df_comments)))

(1996, 25)
(8911, 19)
Lenght of the old 999
Lenght of the new 1362
Lenght of the old 4460
Lenght of the new 6704
(999, 25)
(1362, 25)
(4460, 19)
(6704, 19)
363 new submissions added
2244 new comments added


In [14]:
print(new_submissions.columns)
print(new_comments.columns)

Index(['id', 'subreddit_id', 'title', 'score', 'author', 'author_flair_text',
       'likes', 'archived', 'is_video', 'name', 'over_18', 'pinned',
       'quarantine', 'selftext', 'send_replies', 'num_comments',
       'num_crossposts', 'distinguished', 'created', 'created_utc',
       'comment_limit', 'approved_by', 'ups', 'downs', 'gilded'],
      dtype='object')
Index(['comment_id', '_submission', 'subreddit_id', 'score', 'author',
       'author_flair_text', 'archived', 'name', 'body', 'send_replies',
       'distinguished', 'created', 'created_utc', 'approved_by', 'link_id',
       'parent_id', 'ups', 'downs', 'gilded'],
      dtype='object')


# 3) Export Dataframes to CSV

In [15]:
import os
import sys
import datetime

now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
print(now)

print('reddit_comments_submission_' + str(now) + '.csv')

2018-05-10_1022
reddit_comments_submission_2018-05-10_1022.csv


In [18]:
new_submissions.to_csv('reddit_submissions_wUTC_' + str(now) + '.csv', index=True, header = True)
new_comments.to_csv('reddit_comments_wUTC' + str(now) +'.csv', index=True, header = True)